# Many Variables  GHZ

$c = C$  but no scalings

In [1]:
print('Started!')
from system import *
s1 = system('o-o-x-o-',MMA=True,ManyVariables = True)
#s1 = load_object('GHZ')

Started!
Initializing system o-o-x-o-  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  o-o-x-o-  initialized in 569.7 seconds.


# Effective Hamiltonian Matrix obtain 

In [1]:
n = 0  #exponent

total_indices = s1.variable_index

var('De1', domain='positive')
var('Deg1', domain='positive' , latex_name =r'{{\Delta e}}_{{  1}}^\gamma')
var('De2', domain='positive')
var('Deg2', domain='positive' , latex_name =r'{{\Delta e}}_{{  2}}^\gamma')
var('De3', domain='positive')
var('Deg3', domain='positive' , latex_name =r'{{\Delta e}}_{{  3}}^\gamma')    

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','phi',domain='positive')
var('DEg', domain='positive' , latex_name =r'{{\Delta E}}^\gamma')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')
var('r_f',domain='positive',latex_name =r'r_f')
var('r_g',domain='positive',latex_name =r'r_g')
var('R_f',domain='positive')#ratio  (g_f/g)^2
eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]
        b = b.operands()[1].operands()[term]
        if len(str(b))<10: #we isolated Omega^2 instead of one of the conjugate pairs
            b = s1.eff_hamiltonian[diag,diag].operands()[0].operands()[term]
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De1->Deg1*gamma*C^({n})')._sage_()
        #b = b.subs(De1 = Deg1*gamma*C^n) #sqrt C scaling!?
        
        b = b._mathematica_().ReplaceAll(f'De2->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De2 = Deg2*gamma*C^n) #MAKE ZERO
        
        b = b._mathematica_().ReplaceAll(f'De3->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        #b = b.subs(v = (c*kappa_b*kappa_c)^(1/2) )
        
        

        #Substitutions
        #b = b.subs(c=np.pi,R_f=2.1,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2), phi=phi_val) 
        #b = b.subs(phi=phi_val)

        #b = b._mathematica_().Factor()._sage_()
        #b = b._mathematica_().Together()._sage_()


        b = SR(str(b).replace('Sqrt','sqrt'))
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
    
    h_eff = h_eff.subs(c=C)
    h_eff = MMA_simplify(h_eff)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    var('x')
    h_eff = h_eff.subs(C=1/x)
    h_eff = MMA_simplify(h_eff)
    h_eff = h_eff._mathematica_().Series((x,0,1)).Normal()._sage_()
    h_eff = h_eff.subs(x=1/C)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    eff_hamiltonian_tayl[diag,diag] = h_eff  * Omega^2 / 2

print('\n Done! ')
#show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=3))


eff_hamiltonian_tayl_temp = eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] 



lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number):
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not str(symround(L_matrix[i,j]))==str(0):
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]
        b = L_elem
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De1->Deg1*gamma*C^({n})')._sage_()
        #b = b.subs(De1 = Deg1*gamma*C^n) #sqrt C scaling!?
        
        b = b._mathematica_().ReplaceAll(f'De2->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De2 = Deg2*gamma*C^n) #MAKE ZERO
        
        b = b._mathematica_().ReplaceAll(f'De3->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        
        b = b.subs(c=C)
        
        b = SR(str(b).replace('Sqrt','sqrt'))
        
        eff_lind[lind_op].append(b)
        
print('\n Done!')


eff_hamiltonian_tayl = eff_hamiltonian_tayl_temp

a = eff_hamiltonian_tayl[0,0]
#a = symround( MMA_simplify(a,full=False) ,digits = 14)
b = eff_hamiltonian_tayl[1,1]
#b = symround(MMA_simplify(b,full=False),digits = 14)
c = eff_hamiltonian_tayl[3,3]
#c = symround(MMA_simplify(c,full=False),digits = 14)
d = eff_hamiltonian_tayl[4,4]
#d = symround(MMA_simplify(d,full=False),digits = 14)
e = eff_hamiltonian_tayl[5,5]
#e = symround(MMA_simplify(e,full=False),digits = 14)
f = eff_hamiltonian_tayl[7,7]
#f = symround(MMA_simplify(f,full=False),digits = 14)

NameError: name 's1' is not defined

# Isolate highest powers

In [3]:
a = eff_hamiltonian_tayl[0,0]
#a = symround( MMA_simplify(a,full=False) ,digits = 14)
b = eff_hamiltonian_tayl[1,1]
#b = symround(MMA_simplify(b,full=False),digits = 14)
c = eff_hamiltonian_tayl[3,3]
#c = symround(MMA_simplify(c,full=False),digits = 14)
d = eff_hamiltonian_tayl[4,4]
#d = symround(MMA_simplify(d,full=False),digits = 14)
e = eff_hamiltonian_tayl[5,5]
#e = symround(MMA_simplify(e,full=False),digits = 14)
f = eff_hamiltonian_tayl[7,7]
#f = symround(MMA_simplify(f,full=False),digits = 14)


#SUBSTITUTIONS
phival = 0
R_fval = 1
gamma_fval = gamma
gamma_gval = gamma


#modify abcdef
a = (a + conjugate(a)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()
b = (b + conjugate(b)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()
c = (c + conjugate(c)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()
d = (d + conjugate(d)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()
e = (e + conjugate(e)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()
f = (f + conjugate(f)).subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()

In [6]:
term_list = [a,b,c,d,e,f]
high_c_limit = [  ]

for term in term_list:
    num = term._mathematica_().Numerator().Collect('C')._sage_().operands()[0]
    i = 1
    while num.has(C)==False:
        num = term._mathematica_().Numerator().Collect('C')._sage_().operands()[i]
        i  = i + 1
        print(i)
    try:
        den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[0]
        i=1
        while den.has(C)==False:
            den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[i]
            i= i + 1
    except:
        den= term._mathematica_().Denominator()._sage_().expand()._mathematica_().Collect('C')._sage_().operands()[0]
        i = 1
        while den.has(C)==False:
            den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[i]
            i= i + 1
    num = symround(num,digits=14,show_del=False)
    den = symround(den,digits=14,show_del=False)
    tayl = num/den
    if len(high_c_limit)<=5:
        tayl = MMA_simplify(tayl,full=True)
    high_c_limit.append(tayl)

In [7]:
a = MMA_simplify(symround(high_c_limit[0],digits=14),full=True)
b = MMA_simplify(symround(high_c_limit[1],digits=14),full=True)
c = MMA_simplify(symround(high_c_limit[2],digits=14),full=True)
d = MMA_simplify(symround(high_c_limit[3],digits=14),full=True)
e = MMA_simplify(symround(high_c_limit[4],digits=14),full=True)
f = symround(high_c_limit[5],digits=14)

In [11]:
'''
view = a#.subs(Deg1=DEg,Deg2=DEg,Omega=1,gamma=1)
view = view.subs()
view = symround((view))
show(view._mathematica_().Numerator().Collect('C')._sage_().collect(C).operands()[0])
print(view._mathematica_().Denominator().Collect('C')._sage_().collect(C).operands()[0])
'''

"\nview = a#.subs(Deg1=DEg,Deg2=DEg,Omega=1,gamma=1)\nview = view.subs()\nview = symround((view))\nshow(view._mathematica_().Numerator().Collect('C')._sage_().collect(C).operands()[0])\nprint(view._mathematica_().Denominator().Collect('C')._sage_().collect(C).operands()[0])\n"

In [10]:
show(a)

4*DEg*Omega^2/(C^2*gamma)

In [224]:
op = 2
pos = 0

q = eff_lind[op][pos]

q = q* conjugate(q)

q = q.subs(phi=phival,R_f=R_fval ,gamma_g = gamma_gval , gamma_f = gamma_fval)._mathematica_().Together()._sage_()

term = q

num = term._mathematica_().Numerator().Collect('C')._sage_().operands()[0]
i = 1
while num.has(C)==False:
    num = term._mathematica_().Numerator().Collect('C')._sage_().operands()[i]
    i  = i + 1
    print(i)
try:
    den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[0]
    i=1
    while den.has(C)==False:
        den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[i]
        i= i + 1
except:
    den= term._mathematica_().Denominator()._sage_().expand()._mathematica_().Collect('C')._sage_().operands()[0]
    i = 1
    while den.has(C)==False:
        den = term._mathematica_().Denominator().Collect('C')._sage_().operands()[i]
        i= i + 1
num = symround(num,digits=14,show_del=False)
den = symround(den,digits=14,show_del=False)
tayl = num/den

tayl = MMA_simplify(tayl,full=True)

show(eff_lind_coeff[op], '   .',pos)
show( symround(tayl ))
print(latex(symround(tayl)))


sqrt(kappa_b) '   .' 0

0.062*Omega^2/(C^2*gamma)

\frac{0.062 \, {\Omega}^{2}}{C^{2} {\gamma}}


In [225]:

term =term.subs(kappa_b=1)#,Deg1=1*C^m,Deg2=1*C^m,DEg=1*C^m)
qnum = symround(term._mathematica_().Numerator().Collect('C')._sage_())
qden = symround(term._mathematica_().Denominator().Collect('C')._sage_())
show(qnum)
print(latex(qnum))

0.062*C^6*Omega^2 + 0.016*C^5*Omega^2 + 0.00098*C^4*Omega^2

0.062 \, C^{6} {\Omega}^{2} + 0.016 \, C^{5} {\Omega}^{2} + 0.00098 \, C^{4} {\Omega}^{2}


In [226]:
show(qden)
print(latex(qden))

C^8*gamma + (4*DEg^2 + 8.4)*C^6*gamma + 5.2*C^7*gamma + (2.5*DEg^2 + 3.9)*C^5*gamma + (0.64*DEg^2 + 0.87)*C^4*gamma + (0.086*DEg^2 + 0.11)*C^3*gamma + (0.0063*DEg^2 + 0.0072)*C^2*gamma + (0.00024*DEg^2 + 0.00026)*C*gamma + ((3.8e-6)*DEg^2 + 3.8e-6)*gamma

C^{8} {\gamma} + {\left(4 \, {{{\Delta E}}^\gamma}^{2} + 8.4\right)} C^{6} {\gamma} + 5.2 \, C^{7} {\gamma} + {\left(2.5 \, {{{\Delta E}}^\gamma}^{2} + 3.9\right)} C^{5} {\gamma} + {\left(0.64 \, {{{\Delta E}}^\gamma}^{2} + 0.87\right)} C^{4} {\gamma} + {\left(0.086 \, {{{\Delta E}}^\gamma}^{2} + 0.11\right)} C^{3} {\gamma} + {\left(0.0063 \, {{{\Delta E}}^\gamma}^{2} + 0.0072\right)} C^{2} {\gamma} + {\left(0.00024 \, {{{\Delta E}}^\gamma}^{2} + 0.00026\right)} C {\gamma} + {\left(\left(3.8 \times 10^{-6}\right) \, {{{\Delta E}}^\gamma}^{2} + 3.8 \times 10^{-6}\right)} {\gamma}


# $t_g$ calculation

In [138]:
t_den = (a + e - b - d)._mathematica_().Together()._sage_()

ttden = symround(t_den, show_del=True,digits=14  )
numnum = 0
dennum = 0
'''
while True:
    print(f'numnum={numnum}  ,  dennum={dennum}')
    num = ttden._mathematica_().Numerator().Collect('C')._sage_().operands()[numnum]
    den = ttden._mathematica_().Denominator().Collect('C')._sage_().operands()[dennum]
    tayl = num/den
    taylsimp = MMA_simplify(tayl,full=True)
    retaylsimp = taylsimp  #(taylsimp+conjugate(taylsimp))   #/2
    taylf = retaylsimp._mathematica_().Together()._sage_()
    taylf = symround(taylf,digits=14)
    if str(taylf) == str(0):
        numnum += 1
    else:
        break
'''
        
print('Finished!')

tg = np.pi / ttden

Finished!


In [139]:
show(symround(MMA_simplify(tg,full=True).subs()))

13.*((Deg1^2 + 0.25)*DEg^2 + 0.50*DEg*Deg1 + 0.25*Deg1^2)*(DEg^2 + 0.25)*C*gamma/(((DEg*Deg1 - 0.25)*DEg - 0.25*Deg1)*Omega^2)

# condition 1 calculation

In [140]:
t_den = (a + c - 2*b )._mathematica_().Together()._sage_()

ttden = symround(t_den, show_del=True,digits=14  )

print('Finished!')

cond1 = tg*ttden

Finished!


In [141]:
show(symround(MMA_simplify(cond1,full= not True).subs(),digits =5))

((-3.1416*Deg1^2 - 0.78540)*DEg^2 - 1.5708*DEg*Deg1 - 0.78540*Deg1^2)/((DEg^2*Deg1 - 0.25000*DEg - 0.25000*Deg1)*DEg)

# condition 2 calculation

In [142]:
# f added
t_den = (2*a - 2*b -d + f )._mathematica_().Together()._sage_()
ttden = symround(t_den, show_del=True,digits=14 )

taylnf = ttden
show(symround(taylnf,show_del=False))

cond2 = taylnf*tg
cond2 = MMA_simplify(cond2,full=not True)

-(C^2*DEg^6*Deg1^3*Omega^2 + 0.25*C^2*DEg^6*Deg1*Omega^2 + 0.25*C^2*DEg^5*Deg1^2*Omega^2 + 0.75*C^2*DEg^4*Deg1^3*Omega^2 + 0.19*C^2*DEg^4*Deg1*Omega^2 + 0.31*C^2*DEg^3*Deg1^2*Omega^2 + 0.062*DEg^5*Deg1^2*Omega^2 + 0.12*C^2*DEg^2*Deg1^3*Omega^2 + 0.69*DEg^4*Deg1^3*Omega^2 + 0.016*DEg^5*Omega^2 + 0.20*DEg^4*Deg1*Omega^2 + 0.38*DEg^3*Deg1^2*Omega^2 + 0.34*DEg^2*Deg1^3*Omega^2 + 0.0039*DEg^3*Omega^2 + 0.051*DEg^2*Deg1*Omega^2 + 0.090*DEg*Deg1^2*Omega^2 + 0.043*Deg1^3*Omega^2)/((DEg^2*Deg1^2 + 0.25*DEg^2 + 0.50*DEg*Deg1 + 0.25*Deg1^2)*(DEg^2 + 0.25)*C^3*DEg^3*Deg1*gamma)

In [145]:
show(symround(MMA_simplify(cond2,full=not True),show_del=False))

((-13.*Deg1^2 - 3.2)*C^2*DEg^6*Deg1 + ((-3.2*C^2 - 0.79)*Deg1^2 - 0.20)*DEg^5 + ((-9.4*Deg1^2 - 2.3)*C^2 - 8.6*Deg1^2 - 2.6)*DEg^4*Deg1 + ((-3.9*C^2 - 4.7)*Deg1^2 - 0.049)*DEg^3 + ((-1.6*C^2 - 4.3)*Deg1^3 - 0.64*Deg1)*DEg^2 - 1.1*DEg*Deg1^2 - 0.54*Deg1^3)/((DEg^2*Deg1 - 0.25*DEg - 0.25*Deg1)*C^2*DEg^3*Deg1)

# a bit of processing of the conditions. Use of $D_i = \dfrac{Deg_i}{DEg}$

In [146]:
cond1 = MMA_simplify(cond1,full=True)
show(symround(cond1))
print('\n\n\n')
cond2 = MMA_simplify(cond2,full=True)
show(symround(cond2))

((-3.2*Deg1^2 - 0.79)*DEg^2 - 1.6*DEg*Deg1 - 0.79*Deg1^2)/(((DEg*Deg1 - 0.25)*DEg - 0.25*Deg1)*DEg)

(-0.20*DEg^5 + (((-3.2*DEg^2 - 2.3)*C^2 - 2.6)*DEg^2 - 0.64)*DEg^2*Deg1 + ((-3.2*C^2 - 0.79)*DEg^4 + (-3.9*C^2 - 4.7)*DEg^2 - 1.1)*DEg*Deg1^2 + (((-13.*DEg^4 - 9.4*DEg^2 - 1.6)*C^2 - 8.6*DEg^2 - 4.3)*DEg^2 - 0.54)*Deg1^3 - 0.049*DEg^3)/(((DEg*Deg1 - 0.25)*DEg - 0.25*Deg1)*C^2*DEg^3*Deg1)

In [152]:
var('D1', domain='positive',latex_name='D_1')
var('D2', domain='positive',latex_name='D_2')
cond1sub = cond1.subs(Deg1 = DEg *D1 , Deg2 = DEg *D2 )
cond1sub = MMA_simplify(cond1sub,full=True)
cond2sub = cond2.subs(Deg1 = DEg *D1 , Deg2 = DEg *D2 )
cond2sub = MMA_simplify(cond2sub,full=True)

In [153]:
show(symround(cond1sub))
print('\n\n')
show(symround(cond2sub))

-3.2*D1 + (D1*(-1.6*D1 - 2.3) - 0.79)/(D1*DEg^2 - 0.25*D1 - 0.25)

(-13.*C^2*D1^3*DEg^6 + ((D1*(-9.4*D1 - 3.2) - 3.2)*C^2 + D1*(-8.6*D1 - 0.79))*D1*DEg^4 + (((D1*(-1.6*D1 - 3.9) - 2.3)*C^2 + D1*(-4.3*D1 - 4.7) - 2.6)*D1 - 0.20)*DEg^2 + (D1*(-0.54*D1 - 1.1) - 0.64)*D1 - 0.049)/((D1*DEg^2 - 0.25*D1 - 0.25)*C^2*D1*DEg^2)

In [6]:
print(q)

NameError: name 'q' is not defined

In [7]:
len(high_c_limit)

5

In [84]:
show(symround(a,digits=2))

-2*DEg*Omega^2*sin(2*phi)^4/((-0.75*DEg^2 + (-0.25*DEg^2 - 0.25)*cos(8*phi) + (DEg^2 + 2)*cos(4*phi) - 2.2)*gamma)

In [79]:
show(symround(b))

-0.25*(3*(Deg2^2 + 1.6)*DEg - 4*(Deg2^2 + 0.75)*DEg + (Deg2^2 + 0.25)*DEg + 2*Deg2)*Omega^2/(((Deg2^2 + 0.75)*DEg^2 + (-0.25*Deg2^2 - 0.062)*DEg^2 + (-0.75*Deg2^2 - 1.2)*DEg^2 - DEg*Deg2 - 0.50*Deg2^2 - 1.8)*gamma)

In [81]:
show(symround(MMA_simplify(c._maxima_().demoivre()._sage_().expand(),full=True)))

(-0.25*(Deg2^2 + 0.25)^2*DEg*cos(8*phi) - 0.62*Deg2^3 + (-0.75*Deg2^4 - 2.9*Deg2^2 - 1.5)*DEg + (-0.38*Deg2^3 + (Deg2^4 + Deg2^2 + 0.44)*DEg - 0.16*Deg2)*cos(4*phi) - 0.41*Deg2)*Omega^2/((-2.2*Deg2^4 + (-0.25*DEg^2 - 0.25)*(Deg2^2 + 0.25)^2*cos(8*phi) + (-0.75*Deg2^4 - 2.9*Deg2^2 - 1.5)*DEg^2 + (-1.2*Deg2^2 - 0.81)*DEg*Deg2 - 6.4*Deg2^2 + (2*Deg2^4 + (Deg2^4 + Deg2^2 + 0.44)*DEg^2 + (-0.75*Deg2^2 - 0.31)*DEg*Deg2 + 1.8*Deg2^2 + 0.62)*cos(4*phi) - 3.1)*gamma)

In [77]:
show(symround(d))

-0.25*((Deg1^2 + 0.25)*DEg*cos(8*phi) + 3*(Deg1^2 + 1.6)*DEg - ((4*Deg1^2 + 3)*DEg - Deg1)*cos(4*phi) + Deg1)*Omega^2/(((-0.75*Deg1^2 - 1.2)*DEg^2 - 0.50*DEg*Deg1 - 2.2*Deg1^2 + ((-0.25*Deg1^2 - 0.062)*DEg^2 - 0.25*Deg1^2 - 0.062)*cos(8*phi) + ((Deg1^2 + 0.75)*DEg^2 - 0.50*DEg*Deg1 + 2*Deg1^2 + 1.2)*cos(4*phi) - 3.2)*gamma)

In [78]:
show(symround(e))

(((-0.25*Deg2^2 - 0.062)*Deg1^2 - 0.062*Deg2^2 - 0.016)*DEg*cos(8*phi) + ((-0.75*Deg2^2 - 1.2)*Deg1^2 - 0.62*Deg1*Deg2 - 1.2*Deg2^2 - 1.3)*DEg + (-0.31*Deg1*Deg2 - 0.25*Deg2^2 - 0.10)*Deg1 + (((Deg2^2 + 0.75)*Deg1^2 - 0.38*Deg1*Deg2 + 0.75*Deg2^2 + 0.41)*DEg + (-0.19*Deg1*Deg2 - 0.25*Deg2^2 - 0.094)*Deg1 - 0.078*Deg2)*cos(4*phi) - 0.20*Deg2)*Omega^2*(cos(8*phi) + I*sin(8*phi))*e^(-8*I*phi)/(((-1.2*Deg1^2 - 1.3)*DEg^2 + ((-0.75*Deg1^2 - 1.2)*DEg^2 - 0.50*DEg*Deg1 - 2.2*Deg1^2 - 3.2)*Deg2^2 + ((Deg1^2 + 0.25)*DEg^2 + Deg1^2 + 0.25)*(-0.25*Deg2^2 - 0.062)*cos(8*phi) - 0.20*DEg*Deg1 - 2.9*Deg1^2 + ((-0.62*DEg*Deg1 - 0.62*Deg1^2 - 0.41)*DEg - 1.1*Deg1)*Deg2 + ((0.75*Deg1^2 + 0.41)*DEg^2 + ((Deg1^2 + 0.75)*DEg^2 - 0.50*DEg*Deg1 + 2*Deg1^2 + 1.2)*Deg2^2 - 0.19*DEg*Deg1 + 1.2*Deg1^2 + ((-0.38*DEg*Deg1 - 0.38*Deg1^2 - 0.16)*DEg - 0.50*Deg1)*Deg2 + 0.61)*cos(4*phi) - 3)*gamma)

False